In [4]:
import os, logging
from pyspark.sql import SparkSession
from pyspark.sql.types import * 
import pyspark.sql.functions as sf
import pyspark.errors as pe
from typing import Optional, Union


In [ ]:
SILVER_PATH = '../data/silver/'
BRONZE_PATH = '../data/bronze/'
RAW_PATH = '../raw'
LOG_DIR = '../log'

In [ ]:
logger = logging.getLogger("Silver_Layer_Log")
logger.setLevel(logging.DEBUG)
log_path = os.path.join(LOG_DIR,"silver.log")
fh = logging.FileHandler(log_path)
sh = logging.StreamHandler()
formatter = logging.Formatter("%(asctime)s - %(name)s - %(level)s - %(message)s")
fh.setFormatter(formatter)
sh.setFormatter(formatter)

if not logger.handlers:
    logger.addHandler(fh)
logger.addHandler(sh)


In [7]:
# Create Spark Session
spark = SparkSession.builder.appName("Retail_ETL_Pipeline").\
    config("spark.sql.shuffle.partitions", "4").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/04 17:07:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
# Read bronze files
def load_bronze(table:str):
    file_path = os.path.join(BRONZE_PATH,f"{table}_clean.parquet")
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"No files found in {file_path}")
    df = spark.read.parquet(file_path)
    df.show(5)


+-------------+---------------+
|department_id|department_name|
+-------------+---------------+
|            2|        Fitness|
|            3|       Footwear|
|            4|        Apparel|
|            5|           Golf|
|            6|       Outdoors|
+-------------+---------------+
only showing top 5 rows


In [9]:
# Clean bronze files

In [10]:
# Enrich tables

In [11]:
# Save as silver.parquet